In [107]:
from PIL import Image

In [5]:
image_path = '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps copy.png'
final_path = '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps/'

In [6]:
def crop_images_to_2_rows(file_path, output_folder):

    with Image.open(file_path) as img:
        image_width, image_height = img.size
        # Since there are 8 images per row and 2 rows
        single_image_width = image_width // 8
        single_image_height = image_height // 2
        
        images = []
        for row in range(2):  # two rows
            for col in range(8):  # eight images per row
                left = col * single_image_width
                top = row * single_image_height
                right = (col + 1) * single_image_width
                bottom = (row + 1) * single_image_height
                crop = img.crop((left, top, right, bottom))
                crop_path = f"{output_folder}/image_row{row}_col{col}.png"
                crop.save(crop_path)
                images.append(crop_path)
                
    return images

In [7]:
crop_images_to_2_rows(image_path, final_path)

['/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col0.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col1.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col2.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col3.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col4.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col5.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col6.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row0_col7.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row1_col0.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row1_col1.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row1_col2.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-eps//image_row1_col3.png',
 '/Users/karthik/Desktop/PROCESS/1by4_Basic_GAN_200-

**Cropping Images for 2 rows & 8 columns**

In [160]:
import os
from PIL import Image

def crop_image(image_path, output_folder, num_cols=8, num_rows=2):

    with Image.open(image_path) as img:
        image_width, image_height = img.size
        single_image_width = image_width // num_cols
        single_image_height = image_height // num_rows

        # Create a new directory for the cropped images
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        image_output_folder = os.path.join(output_folder, name)
        if not os.path.exists(image_output_folder):
            os.makedirs(image_output_folder)

        # Crop and save the images
        for row in range(num_rows):
            for col in range(num_cols):
                left = col * single_image_width
                top = row * single_image_height
                right = (col + 1) * single_image_width
                bottom = (row + 1) * single_image_height
                crop = img.crop((left, top, right, bottom))
                crop_path = os.path.join(image_output_folder, f"{name}_row{row}_col{col}{ext}")
                crop.save(crop_path)

def process_images_in_directory(directory_path, output_folder):

    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(directory_path, filename)
            crop_image(image_path, output_folder)

# Set the correct paths
directory_path = '/Users/karthik/Desktop/PROCESS/'
output_folder = '/Users/karthik/Desktop/PROCESS/Cropped/'

# Process all images in the directory
process_images_in_directory(directory_path, output_folder)

**Cropping Images for 4 row & 4 column**

In [25]:
import os
from PIL import Image

def crop_image(image_path, output_folder, num_cols, num_rows):

    with Image.open(image_path) as img:
        image_width, image_height = img.size
        single_image_width = image_width // num_cols
        single_image_height = image_height // num_rows

        # Create a new directory for the cropped images
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        image_output_folder = os.path.join(output_folder, name)
        if not os.path.exists(image_output_folder):
            os.makedirs(image_output_folder)

        # Crop and save the images
        for row in range(num_rows):
            for col in range(num_cols):
                left = col * single_image_width
                top = row * single_image_height
                right = (col + 1) * single_image_width
                bottom = (row + 1) * single_image_height
                crop = img.crop((left, top, right, bottom))
                crop_path = os.path.join(image_output_folder, f"{name}_row{row}_col{col}{ext}")
                crop.save(crop_path)

def process_images_in_directory(directory_path, output_folder, num_cols, num_rows):

    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(directory_path, filename)
            crop_image(image_path, output_folder, num_cols, num_rows)

# Set the correct paths
directory_path = '/Users/karthik/Desktop/PROCESS/'
output_folder = '/Users/karthik/Desktop/PROCESS/Cropped/'

# Parameters for cropping (number of columns and rows)
num_cols = 4
num_rows = 4

# Process all images in the directory with the specified number of columns and rows
process_images_in_directory(directory_path, output_folder, num_cols, num_rows)


Preparing for FID SCORE 

In [28]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import inception_v3
import numpy as np
from scipy.linalg import sqrtm

def get_inception_features(images, model, batch_size=32):
    model.eval()
    features = []
    
    # Transform input images to match Inception's expectations
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    for img_path in images:
        img = Image.open(img_path).convert('RGB')
        img = transform(img).unsqueeze(0)  # Add batch dimension
        with torch.no_grad():
            pred = model(img)
        features.append(pred.squeeze(0).cpu().numpy())
        
    features = np.array(features)
    return features

In [29]:
def calculate_fid(real_features, gen_features):
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = gen_features.mean(axis=0), np.cov(gen_features, rowvar=False)
    
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    
    # Check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [30]:
inception_model = inception_v3(pretrained=True)
inception_model.fc = torch.nn.Identity()  

/Users/karthik/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/karthik/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /Users/karthik/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|████████████████████████████████████████| 104M/104M [00:05<00:00, 20.3MB/s]


<H3>CALCULATING FID SCORE FOR Vanilla GAN Generated Images

**Vanilla GANs 4 Layer   Eps = 200**

In [31]:
import os

# Assuming you have a directory with real images and another with generated images
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Vanilla_GAN_with_4_Layers/VG-4Layers-200eps/'

# List the paths to the real images
real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

# Make sure there are 2176 real images
assert len(real_image_paths) == 2176, "The number of real images should be 2176"

# List the paths to the generated images
generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

# Make sure there are 16 generated images
assert len(generated_image_paths) == 16, "The number of generated images should be 16"


In [32]:
# Calculate features
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

# Calculate FID
fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 496.7845755492306


**Vanilla GANs 4 Layer   Eps = 250**

In [33]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Vanilla_GAN_with_4_Layers/VG-4Layer-250eps/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [34]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 497.99034096661757


**Vanilla GANs 4 Layer   lr-0.0002 - 200 Eps**

In [35]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Vanilla_GAN_with_4_Layers/VG-4Layers-lr-0.0002-200eps/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [36]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 488.18345876653063


**1 by 4 - Vanilla GANs - 4Layers - 200eps**

In [37]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Vanilla_GAN_with_4_Layers/1by4-VG-4Layers-200eps/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [38]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 422.98492524014284


**1 by 4 - Vanilla GANs- 64X64 - 200 Eps - lr = 2**

In [39]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Vanilla_GAN_with_4_Layers/1by4-VG-64X64-200eps-lr_2/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [40]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 419.21803184852365


**1 by 4 - Vanilla GANs- 64X64 - 200 Eps - lr = 3**

In [41]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Vanilla_GAN_with_4_Layers/1by4-VG-64X64-200eps-lr_3/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [42]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 427.8725253176444


**2 by 2 Vanilla-GAN - Increased Sample - 2 by 2 Resized and Rotated Dataset**

In [168]:
real_images_dir = '/Users/karthik/GANS/FiguresStacked-2X2/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Increased-Sample/2by2-VG-Increased-Sample/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [169]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 478.16585930439976


**4 by 4 Vanilla-GAN - Increased Sample - 4 by 4 Resized and Rotated Dataset**

In [170]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/Vanilla_GANs/Increased-Sample/Vanilla-GAN-4by4-Increased-Sample/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [171]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 448.3321029119707


<H3>CALCULATING FID SCORE FOR C-GAN Generated Images

**C-GAN using ANN 4 Layer G & D - BCE Loss - eps = 250 Z_dim = 112**

In [49]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_4_Layers_BCE_Loss/C-GAN-using-ANN-4-Layer-G&D-BCELoss- eps-250Z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [50]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 507.7035363649482


**C-GAN using ANN 4 Layer G & D BCE Loss EPS = 200**

In [51]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_4_Layers_BCE_Loss/C-GAN-using-ANN-4-Layer-G&D-BCELoss-200 EPS/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [52]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 474.7411320123402


**C-GAN using ANN 4 Layer G & D BCE Loss EPS = 250**

In [53]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_4_Layers_BCE_Loss/C-GAN-using-ANN-4-Layer-G&D-BCELoss-250-EPS/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [54]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 509.0256116218969


**C-GAN using ANN 4 Layer G & D BCE Loss Eps = 200 Z_dim = 112**

In [55]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_4_Layers_BCE_Loss/C-GAN-using-ANN-4-Layer-G&D-BCELoss-eps-200Z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [56]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 502.98547592438734


**C-GAN using ANN 5 Layer G & D BCELoss - Eps = 200 - z_dim = 112 TUNING**

In [57]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_5_Layers_BCE_Loss/C-GAN-using-ANN-5-Layer-G&D-BCELoss- eps200-z_dim -112TUNING/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [58]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 509.50546782793595


**C-GAN using ANN 5 Layer G & D - BCELoss - Eps = 200 - z_dim = 112**

In [59]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_5_Layers_BCE_Loss/C-GAN-using-ANN-5-Layer-G&D-BCELoss-eps200-z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [60]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 499.2763402028513


**C-GAN using ANN 5 Layer G & D BCELoss Eps = 250 - z_dim = 112**

In [61]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_5_Layers_BCE_Loss/C-GAN-using-ANN-5-Layer-G&D-BCELoss-eps250-z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [62]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 512.1563970357131


**C-GAN using ANN 5 Layer G with BN of 3Layers BCELoss**

In [63]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_5_Layers_BCE_Loss/C-GAN-using-ANN-5-Layer-G-with-BN-of-3Layers-BCELoss/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [64]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 546.3369696526111


**C-GAN using ANN 5 Layer G & D BCELoss - EPS = 250**

In [65]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_5_Layers_BCE_Loss/C-GAN_using_ANN_5_Layer_G&D-BCELoss-250EPS/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [66]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 521.9359131127594


**C-GAN using ANN 5 Layer G with BN of 3 Layers BCEWithLogitsLoss**

In [67]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_5_Layers_BCE_Loss/C-GAN_using_ANN_5_Layer_G_with_BN_of_3_Layers-BCEWithLogitsLoss/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [68]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 534.0923662798457


**C-GAN using ANN with 5 Layer G with BN of 1Layers MSELoss**

In [69]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_MSE_Loss/C-GAN_using-ANN-5-Layer-G-with-BN-of-1Layers-MSELoss/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [70]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 488.8694451641635


**C-GAN using ANN with 5 Layer G with BN of 2Layers MSELoss**

In [71]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_MSE_Loss/C-GAN_using-ANN-5-Layer-G-with-BN-of-2-Layers -MSELoss/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [72]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 490.8425120849204


**C-GAN using ANN with 5 Layer G with BN of 3 Layers MSELoss**

In [73]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_MSE_Loss/C-GAN_using-ANN-5-Layer-G-with-BN-of-3-Layers -MSELoss/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [74]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 491.61758050569006


**C-GAN using ANN with 5 Layer G with No BN Layers MSELoss**

In [75]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_MSE_Loss/C-GAN_using_ANN_5_Layer_G D_with_NO_BN_Layers-MSE Loss/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [76]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 531.6132213796026


**C-GAN using ANN with 5 Layer G with BN of 1Layers MSELoss Tuning**

In [77]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_MSE_Loss/C-GANusingANN5-Layer-G-with-BN-of-2-Layers-MSELoss-TUNING/**C-GAN using ANN with 5 Layer G with BN of 1Layers MSELoss**'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [78]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 491.2249674784682


**C-GAN using ANN 6 Layer G & 5 layer D - BCELoss - Eps = 200 - z_dim = 112**

In [79]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_6_Layers/C-GANusingANN-6-Layer-G&5layerD-BCELoss-eps200-z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [80]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 481.02084647738286


**C-GAN using ANN 6 Layer G & D - BCELoss - Eps = 200 - z_dim = 112**

In [81]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_6_Layers/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [82]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 494.44503296931066


**C-GAN using ANN 6 Layer G & D - BCELoss - Eps = 200 - z_dim = 112 Tuning**

In [83]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_6_Layers/C-GANusingANN-6-Layer-G&D-BCELoss-eps200-z_dim-112 Tuning/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [84]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 492.08403720514593


**C-GAN using ANN 7 - Layer G & 6 - layer - D - BCELoss - Eps = 250 - z_dim = 112**

In [85]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_7_Layers/C-GANusingANN-7-LayerG&6-layer-D-BCELoss-eps250-z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [86]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 502.9398728074666


**C-GAN using ANN 7 - Layer G & D - BCELoss - Eps = 250 - z_dim = 112 - COMBO - 2**

In [87]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_7_Layers/C-GANusingANN-7-LayerG&D-BCELoss-eps200-z_dim-112-COMBO-2/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [88]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 493.31799868400844


**C-GAN using ANN 7 - Layer G & D - BCELoss - Eps = 250 - z_dim = 112 COMBO - 1 Tuning**

In [89]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_7_Layers/C-GANusingANN-7-LayerG&D-BCELoss-eps250-z_dim-112-COMBO-1-Tuning/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [90]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 516.1787562921003


**C-GAN using ANN 7 - Layer G & D - BCELoss - Eps = 250 - z_dim = 112 COMBO - 1**

In [91]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_7_Layers/C-GANusingANN-7-LayerG&D-BCELoss-eps250-z_dim-112-COMBO-1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [92]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 473.150113626764


**C-GAN using ANN - 8 Layer G & D BCELoss Eps = 250 - z_dim = 112 COMBO - 2 Tuning**

In [94]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_8_Layers/C-GAN-using-ANN-8-Layer-G&D-BCELoss- eps 250-z_dim-112COMBO-2Tuning/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [95]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 490.2710705366761


**C-GAN using ANN - 8 Layer G & D BCELoss Eps = 250 - z_dim = 112 COMBO - 2**

In [96]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_8_Layers/C-GAN_using-ANN-8-Layer-G&D-BCELoss-eps 250-z_dim-112-COMBO-2-Copy1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [97]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 480.23128042229456


**C-GAN using ANN - 8 Layer G & D BCELoss Eps = 250 - z_dim = 112 COMBO - 1 Tuning**

In [98]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_8_Layers/C-GAN_using_ANN-8-Layer-G&D-BCELoss-eps-250-z_dim-112-COMBO-1-Tuning/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [99]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 516.9741324275257


**C-GAN using ANN - 8 - Layer - G & D - BCELoss - Eps = 250 - z_dim = 112 - COMBO-2**

In [100]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_8_Layers/C-GAN_using_ANN-8-Layer-G&D-BCELoss-eps-250-z_dim-112-COMBO-2/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [101]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 486.6516508494069


**C-GAN using ANN - 8 Layer - G & D - BCELoss - Eps = 250 - z_dim = 112 - COMBO - 3**

In [102]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_8_Layers/C-GAN_using_ANN-8-Layer-G&D-BCELoss-eps250-z_dim-112-COMBO-3-Copy1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [103]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 511.3109806278178


**C-GAN using ANN - 8 Layer - G & D - BCELoss - Eps = 250 - z_dim = 112 - COMBO - 3**

In [104]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/C-GANs/C-GAN_8_Layers/C-GAN_using_ANN_8-Layer-G&D-BCELoss-eps 250-z_dim-112-COMBO-2-Copy2/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [105]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 484.149182822307


<H3>CALCULATING FID SCORE FOR DC-GAN Generated Images

**1by4 - 128X128 Image Resolution - Residual Block - lr = 3**

In [109]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/1by4-128X128-Residual-Block-lr-3/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [110]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 306.99795392703925


**1by4 - Residual Block - Filters = 128 - EPS = 25**

In [111]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/1by4-ResBlock-128-Filters-25eps/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [112]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 366.75844544403384


**1by4 - Residual Block - 128X128 Image Resolution - lr = 2**

In [113]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/1by4-ResBlock-128X128-lr-2 /'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [114]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 368.8807606005696


**1by4 - Residual Block - Image Resolution 64X64 - 1**

In [115]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/1by4-ResBlock-64X64-1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [116]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 339.48543465349184


**1by4 - Residual Block - Image Resolution 64X64 - 2**

In [117]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/1by4-ResBlock-64X64-2/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [118]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 332.74480486823325


**1by4 - Residual Block - Image Resolution 64X64 - 3**

In [119]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/1by4-ResBlock-64X64-3/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [120]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 339.48543486862263


**1 by 4 - Residual Block - Image Resolution 128X128 - Combo -1**

In [121]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/1by4-Residual-Block/RB-128X128-Combo-1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [122]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 314.0494195522597


**4by4 - Image Resolution - 128X128 - method - 1**

In [123]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/4by4-Topo/4by4-128X128-method-1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [124]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 423.28431534258493


**4by4 - Image Resolution - 64X64**

In [125]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/4by4-Topo/4by4-64x64-1/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [126]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 448.69062650417584


**4by4 - Image Resolution - 256X256**

In [127]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/4by4-Topo/4by4-topo-DC-GAN-256X256/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [128]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 325.47247543368235


**1by4 - Image Resolution 64X64 - EPS = 25 - Batch Size = 64**

In [129]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/Convo-Layers-with-combos/1by4-64X64-25Eps-bs-64/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [130]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 345.446282594336


**1by4 - EDGE Processed Topography - Kernel Degree = 6 - 64 X 64 - EPS = 25**

In [131]:
real_images_dir = '/Users/karthik/GANS/1by4-EDGE-Topography/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/Convo-Layers-with-combos/1by4-EDGE-6-Topo-64X64-eps-25 /'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [132]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 273.1905145542704


**1by4 - EDGE Processed Topography - Kernel Degree = 8 - 64 X 64 - EPS = 25**

In [133]:
real_images_dir = '/Users/karthik/GANS/1by4-Edge-8Degree/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/DC-GANs/Convo-Layers-with-combos/1by4-EDGE-8-Topo-64X64-eps25/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [134]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 276.3324416618382


<H3>CALCULATING FID SCORE FOR Wassertein-GAN Generated Images

**1 by 4 topo - Image Resolution 64 X 64 - lr = 3 - z_dim = 100**

In [135]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/1by4-topo-64X64-lr-3-z_dim-100/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [136]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 322.1146314623371


**1 by 4 - topo - Image Resolution 64 X 64 - lr = 3 - z_dim = 112 - EPS = 40**

In [137]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/1by4-topo-64X64-lr-3-z_dim-112-EPS-40/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [138]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 308.46295284788675


**1 by 4 topo - Image Resolution 64 X 64 - lr = 3 - z_dim = 112**

In [139]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/1by4-topo-64X64-lr-3-z_dim-112/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [140]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 316.8117227020753


**1 by 4 topo - Image Resolution 64 X 64 - lr = 2 - z_dim = 112 - EPS = 40**

In [141]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/1by4_topo_64X64_lr_2_z_dim_112_EPS-40/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [142]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 306.24041750465994


**1 by 4 topo - Image Resolution 64X64 - lr = 3 - z_dim = 112 - EPS = 40 - Filters = 128**

In [143]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/1by4_topo_64X64_lr_3_z_dim_112_EPS-40_128Filters/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [144]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 336.02679792426375


**1 by 4 topo - Image Resolution 64X64 - lr = 3 - z_dim = 112 - EPS = 40 - Filters = 96**

In [157]:
real_images_dir = '/Users/karthik/GANS/1by4-TOPO-Resized/bio_materials/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/1by4_topo_64X64_lr_3_z_dim_112_EPS-40_96Filters/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [158]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 353.6806062680671


**4 by 4 - Image Resolution 64 X 64 - lr = 2**

In [147]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/4by4-64X64-lr2/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [148]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 419.3718660064734


**4 by 4 - Image Resolution 64 X 64 - lr = 3 - EPS = 30**

In [149]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/4by4-64X64-lr3-EPS-30/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [150]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 422.73698421597624


**4 by 4 - Image Resolution 64X64 - lr = 3 - EPS = 40 - Filter = 128**

In [151]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/4by4-64X64-lr3-EPS-40-Filter-128/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [152]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 407.6240499460357


**4 by 4 - Image Resolution 64X64 - lr = 3 - EPS = 40**

In [153]:
real_images_dir = '/Users/karthik/GANS/4by4-TOPO-24X224/renamed_images/'
generated_images_dir = '/Users/karthik/Desktop/Preprocessing_Image_for_FID_SCORE/WGAN/4by4-64X64-lr3-EPS-40/'

real_image_paths = [os.path.join(real_images_dir, image) for image in os.listdir(real_images_dir) if image.endswith('.png')]

assert len(real_image_paths) == 2176, "The number of real images should be 2176"

generated_image_paths = [os.path.join(generated_images_dir, image) for image in os.listdir(generated_images_dir) if image.endswith('.png')]

assert len(generated_image_paths) == 16, "The number of generated images should be 16"

In [154]:
real_features = get_inception_features(real_image_paths, inception_model)
gen_features = get_inception_features(generated_image_paths, inception_model)

fid_score = calculate_fid(real_features, gen_features)
print('FID score:', fid_score)

FID score: 427.8682692488611
